In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
webpage = 'https://es.wikipedia.org/wiki/Anexo:Municipios_de_Espa%C3%B1a_por_poblaci%C3%B3n'

req = Request(webpage, headers={'User-Agent': 'Mozilla/5.0'})
raw_web = urlopen(req, timeout=10).read()

In [3]:
soup = BeautifulSoup(raw_web, 'html.parser')
tables = soup.find_all('table')
titulos_tablas = soup.find_all('span', attrs={"class": "mw-headline"})

In [4]:
print('Títulos de tablas: ', len(tables))
print('Tablas: ', len(tables))

Títulos de tablas:  7
Tablas:  7


In [5]:
head = []
rows_raw = []
for table in tables:
    
    head = table.find_all('th') #We skip the first header as we saw we don't want it
    table_rows = table.find_all('tr')[1:]
    
    for fila in table_rows:
        cols = []
        for col in fila.find_all('td'):
            cols.append(col.text.strip())
            
        rows_raw.append(cols)
    

In [6]:
rows_raw[0]

['1', 'Madrid', '3 305 408', 'Comunidad de Madrid\xa0Comunidad de Madrid']

In [7]:
#Limpieza de cada fila

for row in rows_raw:
    
    if len(row) == 4:
        row.append(row[3])
        
    row[0] = int(row[0])+1000
    row[2] = int("".join(str(row[2]).split(" ")))
    row[3] = row[3].split("\xa0")[0]
    row[4] = row[4].split("\xa0")[0]
    
rows_raw[0]

[1001, 'Madrid', 3305408, 'Comunidad de Madrid', 'Comunidad de Madrid']

In [8]:
for fila in rows_raw:
    
    if len(fila) != len(head):
        print('Cols', len(fila))
        print(fila)
        print('---------------')

In [9]:
all_data = pd.DataFrame(rows_raw, columns=['ID', 'City', 'Population', 'Region', 'State'])

In [10]:
# Generar la tabla State

state_id = 1000
states_raw = all_data['State'].drop_duplicates()

states_list = []

for state in states_raw:
    
    states_list.append({"id": state_id, "name": state, "country": "ES"})
    
    state_id += 1

states = pd.DataFrame(states_list, columns = states_list[0].keys())

states.head()

,id,name,country
0,1000,Comunidad de Madrid,ES
1,1001,Cataluña,ES
2,1002,Comunidad,ES
3,1003,Andalucía,ES
4,1004,Aragón,ES


In [11]:
# Generar la tabla region

region_id = 1000
regions_raw = all_data[['Region','State']].drop_duplicates().reset_index(drop=True)

regions_list = []

for i in regions_raw.index:
    
    region = regions_raw.iloc[i].to_dict()
    
    state_id = states[states['name'] == region['State']].iloc[0]['id']
    
    regions_list.append({"id":region_id, "stateID": state_id, "name": region['Region']})
    
    region_id += 1
    
regions = pd.DataFrame(regions_list, columns=regions_list[0].keys())

regions.head()

,id,stateID,name
0,1000,1000,Comunidad de Madrid
1,1001,1001,Barcelona
2,1002,1002,Valencia
3,1003,1003,Sevilla
4,1004,1004,Zaragoza


In [12]:
# Generar la tabla poblaciones

cities_list = []

for ciudad in rows_raw:
    
    region_id = regions[regions['name'] == ciudad[3]].iloc[0]['id']
    
    cities_list.append({"id": ciudad[0], "regionID": region_id, "name": ciudad[1], "population": ciudad[2]})
    

cities = pd.DataFrame(cities_list, columns=cities_list[0].keys())

cities.head()

,id,regionID,name,population
0,1001,1000,Madrid,3305408
1,1002,1001,Barcelona,1636732
2,1003,1002,Valencia,789744
3,1004,1003,Sevilla,684234
4,1005,1004,Zaragoza,675301


Por último solo es necesario iterar sobre cada tabla con un INSERT INTO para agregar a las tablas SQL